# colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/github/ids

/content/drive/MyDrive/github/ids


# git

In [1]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!git add

In [11]:
!git commit -m ""

[main c617ea6] added colab
 1 file changed, 1 insertion(+)
 create mode 100644 colab.ipynb


In [12]:
!git push

Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 1.04 KiB | 31.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/pro1710/ids.git
   acea846..c617ea6  main -> main


# code

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import random
%matplotlib inline

In [14]:
import importlib
import ids_common
importlib.reload(ids_common)

<module 'ids_common' from '/content/drive/MyDrive/github/ids/ids_common.py'>

In [15]:
path_to_dataset = './dataset/ML-EdgeIIoT-dataset.csv'
df = pd.read_csv(path_to_dataset, low_memory=False)

In [35]:
to_drop_columns = ['frame.time',
                'ip.src_host',
                'ip.dst_host',
                'arp.src.proto_ipv4',
                'arp.dst.proto_ipv4',
                'http.file_data',
                'http.request.full_uri',
                'icmp.transmit_timestamp',
                'http.request.uri.query',
                'tcp.options',
                'tcp.payload',
                'tcp.srcport',
                'tcp.dstport',
                'udp.port',
                'mqtt.msg']


to_drop_1_unique = ['icmp.unused',
                    'http.tls_port',
                    'dns.qry.type',
                    'dns.retransmit_request_in',
                    'mqtt.msg_decoded_as',
                    'mbtcp.len',
                    'mbtcp.trans_id',
                    'mbtcp.unit_id',
                    'mqtt.conack.flags']


to_fix_columns = ['mqtt.topic',
                'mqtt.protoname',
                'dns.qry.name.len',
                'http.request.method',
                'http.referer',
                'http.request.version']


# to drop
# dns.qry.name.len
# dns.retransmit_request


df.loc[(df['dns.qry.name.len'] == '0') | (df[col] == '0x00000000'), col] = '0'

#######################################

for col in to_fix_columns:
    df.loc[(df[col] == '0.0') | (df[col] == '0x00000000'), col] = '0.0'
    # encode_text_dummy(df, col)

stats = []

for col in df.columns:
    uniques = df[col].unique()
    num_uniques = len(uniques)
    stats.append((col, df[col].dtype, num_uniques))

print(f'SHAPE: {df.shape}')

num_rows = df.shape[0]

for i, sts in enumerate(sorted(stats, key=lambda x: x[2])):
    c, t, u = sts
    dropped = '------------'
    if c in to_drop_columns:
        dropped = 'to_drop_columns'
        continue
    elif c in to_drop_1_unique:
        dropped = 'to_drop_1_unique'
        continue


    print(f'{i:<4}{c:<32}{str(t):<12}{u:>6} [{dropped:^24}]')

    if u < 10:
        # val_count = df[c].value_counts().sort_values()
        val_count = df.groupby('Attack_label')[c].value_counts()
        print(val_count)
    print('-'*80)

SHAPE: (157800, 63)
8   arp.hw.size                     float64          2 [      ------------      ]
Attack_label  arp.hw.size
0             0.0             24259
              6.0                42
1             0.0            131967
              6.0              1532
Name: arp.hw.size, dtype: int64
--------------------------------------------------------------------------------
9   http.response                   float64          2 [      ------------      ]
Attack_label  http.response
0             0.0               24301
1             0.0              126280
              1.0                7219
Name: http.response, dtype: int64
--------------------------------------------------------------------------------
10  tcp.connection.fin              float64          2 [      ------------      ]
Attack_label  tcp.connection.fin
0             0.0                    21701
              1.0                     2600
1             0.0                   126924
              1.0               

In [ ]:




    df.drop(to_drop_columns, axis=1, inplace=True)
    print(f'drop: NA: {df.isnull().sum().sum()}, DUPS: {df.duplicated().sum()}', )

    to_drop_1_unique = ['icmp.unused',
                    'http.tls_port',
                    'dns.qry.type',
                    'dns.retransmit_request_in',
                    'mqtt.msg_decoded_as',
                    'mbtcp.len',
                    'mbtcp.trans_id',
                    'mbtcp.unit_id',
                    'mqtt.conack.flags']

    df.drop(to_drop_1_unique, axis=1, inplace=True)
    print(f'drop: NA: {df.isnull().sum().sum()}, DUPS: {df.duplicated().sum()}', )

    df.dropna(axis=0, how='any', inplace=True)
    print(f'dropna: NA: {df.isnull().sum().sum()}, DUPS: {df.duplicated().sum()}', )

    to_fix_columns = ['mqtt.topic',
                    #   'mqtt.conack.flags',
                      'mqtt.protoname',
                      'dns.qry.name.len',
                      'http.request.method',
                      'http.referer',
                      'http.request.version']

    for col in to_fix_columns:
        df.loc[(df[col] == '0.0') | (df[col] == '0x00000000'), col] = '0'
        encode_text_dummy(df, col)


    df.drop_duplicates(subset=None, keep='first', inplace=True)
    print(f'drop_duplicates: NA: {df.isnull().sum().sum()}, DUPS: {df.duplicated().sum()}', )

    df['Attack_type'] = df['Attack_type'].astype('category')

    print('#'*80)
    print('TARGET: "Attack_label"')
    print(df['Attack_label'].value_counts())
    print('-'*80)
    print('TARGET: "Attack_type"')
    print(df['Attack_type'].value_counts())
    print('#'*80)

    return df